In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
pos_data = []
for filename in os.listdir('/content/drive/MyDrive/Train_concrete_images/Positive'):
  pos_data.append(filename)
neg_data = []
for filename in os.listdir('/content/drive/MyDrive/Train_concrete_images/Negative'):
  neg_data.append(filename)

In [ ]:
len(pos_data), len(neg_data)

(18000, 18000)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,BatchNormalization,Dropout,Activation ,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import BinaryCrossentropy
import PIL
from keras.callbacks import EarlyStopping
from keras import optimizers

In [ ]:
def cnn(n,num_filters,ker_size_input,ker_size,activ,pool,num_nodes,filter_org,bn_vs_dp,bn=0,dp='no'):  

  model = Sequential()
  model.add(Conv2D(num_filters, (ker_size_input, ker_size_input), input_shape=(227,227,3)))
  model.add(Activation(activ))
  model.add(MaxPooling2D(pool_size=(pool, pool)))

  for i in range(1,n):
  
    if filter_org=='same':
      num_filters= num_filters
    elif filter_org=='double':
      num_filters*=2
    else:
      num_filters//=2
      
        
    model.add(Conv2D(num_filters, (ker_size, ker_size)))

    if bn:
        model.add(BatchNormalization())
        
    model.add(Activation(activ))
    model.add(MaxPooling2D(pool_size=(pool, pool)))
      
  model.add(Flatten())
  model.add(Dense(num_nodes))

  
  if bn_vs_dp == 'dp':
    model.add(Dropout(dp))
  else:
    model.add(BatchNormalization())

  model.add(Activation(activ))
      
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
def train():
  default_hyperparams = dict(
    bn=0,
    num_filters=64,
    fliter_org='same',
    dropout=0,
    data_aug=0,
    learning_rate=0.01,
    epochs=5,
    activ="ReLU",
    optimizer = 'Adam',
    batch_size=32,
    ker_size_input=5,
    bn_vs_dp='bn'
  )

    
    
  wandb.init(config = default_hyperparams)
  config = wandb.config

  wandb.run.name = 'bn='+str(config.bn)+'num_filters='+str(config.num_filters)+'_'+'filter_org='+str(config.filter_org)+'_dp='+str(config.dropout)+'_data_aug='+str(config.data_aug)+'_lr='+str(config.learning_rate)+'_optimizer='+str(config.optimizer)+'_batch_size='+str(config.batch_size)+'_epochs='+str(config.epochs)+'_activation='+str(config.activ)+'_ker_size='+str(config.ker_size_input)+'_bn_vs_dp'+str(config.bn_vs_dp)
  wandb.run.save()

  train_data_dir= '/content/drive/MyDrive/Train_concrete_images'

  batch_size=config.batch_size

  if config.data_aug:
      
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45, 
    width_shift_range=.15, 
    height_shift_range=.15, 
    horizontal_flip=True, 
    zoom_range=0.5,
    validation_split=0.1) 

  else:
      
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1) # set validation split
  
    

  train_it = train_datagen.flow_from_directory(
      train_data_dir,
      target_size=(227,227),
      batch_size=batch_size,
      class_mode='binary',
      subset='training') # set as training data

  val_it = train_datagen.flow_from_directory(
      train_data_dir, # same directory as training data
      target_size=(227,227),
      batch_size=batch_size,
      class_mode='binary',
      subset='validation') # set as validation data
  
  # Your model here ...
  model = cnn(4,config.num_filters,config.ker_size_input,3,config.activ,2,1024,config.filter_org,config.bn_vs_dp,config.bn,config.dropout)

  if config.optimizer == 'Adam':
    model.compile(optimizers.Adam(learning_rate=config.learning_rate, decay=1e-6), loss=BinaryCrossentropy(), metrics='acc')
  else:              #elif config.optimizer == 'rmsprop':
    model.compile(optimizers.RMSprop(learning_rate=config.learning_rate, decay=1e-6), loss=BinaryCrossentropy(), metrics='acc')
 
  
  model.fit(
  train_it,
  steps_per_epoch = train_it.samples //batch_size,
  validation_data = val_it, 
  validation_steps = val_it.samples // batch_size,
  epochs = config.epochs,
  callbacks=[WandbCallback(data_type='image',validation_data = val_it,verbose=1),EarlyStopping(patience=10,restore_best_weights=True)])

In [ ]:
sweep_config = {
   #'program': train(),
    'method': 'bayes',         
    'metric': {
        'name': 'val_accuracy',     
        'goal': 'maximize'      
    },
    'parameters': {
        'learning_rate': {
            'values': [1e-3,1e-4]
        },
        'activ': {
            'values': ['relu']
        },
        'bn': {
            'values': [0,1]
        },
        'num_filters': {
            'values': [32, 64, 128]
        },
        'filter_org': {
            'values': [ 'same']
        },
        'epochs': {
            'min': 3,
	          'max': 5
        },
        'dropout': {
            'values': [0, 0.2, 0.3, 0.1, 0.4, 0.5]
        },
        'data_aug': {
            'values': [0,1]
        },
        'optimizer' : {
            'values': ['Adam','rmsprop']
        },

        'batch_size': {
            'values': [32, 64]
        },
        'ker_size_input':{
            'values':[3, 5, 7]
        },
        'bn_vs_dp':{
            'values':['bn', 'dp']
        }

        }
    }



In [ ]:
! pip install wandb

     |████████████████████████████████| 1.8MB 2.9MB/s 
     |████████████████████████████████| 174kB 18.8MB/s 
     |████████████████████████████████| 133kB 18.7MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=34265524128589f4a93709b2d7b6e0bd6b4dda659180db2fc596fcbf8d89d670
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=88491ef7eaae4e3e64c78464ef258ed7f02bd89a8422a2c627a7266f3660835e
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.login(key='d1ceb7eee4b7974dd2a8169c606b3c0d311c3276')

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_id = wandb.sweep(sweep_config,project='BTP')

Create sweep with ID: j8fefv6v
Sweep URL: https://wandb.ai/dl_assgn/BTP/sweeps/j8fefv6v


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 7w757cwb with config:
wandb: 	activ: relu
wandb: 	batch_size: 32
wandb: 	bn: 0
wandb: 	bn_vs_dp: bn
wandb: 	data_aug: 1
wandb: 	dropout: 0.2
wandb: 	epochs: 3
wandb: 	filter_org: same
wandb: 	ker_size_input: 3
wandb: 	learning_rate: 0.0001
wandb: 	num_filters: 32
wandb: 	optimizer: Adam
wandb: Currently logged in as: dl_assgn (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Found 32400 images belonging to 2 classes.
Found 3600 images belonging to 2 classes.
Epoch 1/3
1012/1012 [==============================] - 22098s 22s/step - loss: 0.1453 - acc: 0.9459 - val_loss: 0.0731 - val_acc: 0.9741
Epoch 00000: val_loss improved from inf to 0.07313, saving model to /content/wandb/run-20210604_131358-7w757cwb/files/model-best.h5
Epoch 2/3
1012/1012 [==============================] - 558s 552ms/step - loss: 0.0491 - acc: 0.9850 - val_loss: 0.6917 - val_acc: 0.7949
Epoch 3/3
1012/1012 [==============================] - 552s 545ms/step - loss: 0.0408 - acc: 0.9882 - val_loss: 0.1423 - val_acc: 0.9632


epoch,2
loss,0.04128
acc,0.98851
val_loss,0.14235
val_acc,0.96317
_runtime,23265
_timestamp,1622835703
_step,2
best_val_loss,0.07313
best_epoch,0


epoch,▁▅█
loss,█▂▁
acc,▁▇█
val_loss,▁█▂
val_acc,█▁█
_runtime,▁▅█
_timestamp,▁▅█
_step,▁▅█


...

epoch,2
loss,0.01109
acc,0.99666
val_loss,0.0471
val_acc,0.9841
_runtime,591
_timestamp,1622836318
_step,2
best_val_loss,0.01571
best_epoch,0


epoch,▁▅█
loss,█▂▁
acc,▁▇█
val_loss,▁██
val_acc,█▂▁
_runtime,▁▄█
_timestamp,▁▄█
_step,▁▅█


wandb: Agent Starting Run: nykrr8xj with config:
wandb: 	activ: relu
wandb: 	batch_size: 64
wandb: 	bn: 0
wandb: 	bn_vs_dp: dp
wandb: 	data_aug: 1
wandb: 	dropout: 0.3
wandb: 	epochs: 4
wandb: 	filter_org: same
wandb: 	ker_size_input: 3
wandb: 	learning_rate: 0.001
wandb: 	num_filters: 32
wandb: 	optimizer: Adam


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Found 32400 images belonging to 2 classes.
Found 3600 images belonging to 2 classes.
Epoch 1/4
351/506 [===================>..........] - ETA: 2:21 - loss: 0.3218 - acc: 0.8503